## 前処理

1. データ結合する2つのファイルをそれぞれ整形  
   - `cleaned_output.csv`（売上・年月・終了日データ）  
     - 不要な列の削除  
     - 日付の整形（終了日を `YYYY/MM/DD` に統一）  
   - `4gamer_output_correct.csv`（発売日・備考データ）  
     - 元の `4gamer_output.csv` に含まれていた欠損タイトルを手動で補完済み  
     - 必要な列の整形とクリーンアップを実施

2. タイトル列をキーにして、2つのファイルを内部結合

### 1.1. ライブラリとCSVの読み込み

In [27]:
import pandas as pd
from decimal import Decimal

# ファイル読み込み（パスを必要に応じて調整）
file_path = "full_data_with_cleaned_titles.csv"
df = pd.read_csv(file_path)

### 1.2. タイトル整形と不要列削除

In [28]:
# タイトル列をクリーンタイトルで置き換え
df["タイトル"] = df["クリーンタイトル"]

# 不要な列（クリーンタイトル、関連ジャンル）を削除
df = df.drop(columns=["クリーンタイトル", "関連ジャンル"])

### 1.3. サービス終了日の整形

In [29]:
# サービス終了日を datetime に変換し、"YYYY/MM/DD" 形式に整形
df["サービス終了日"] = pd.to_datetime(df["サービス終了日"], errors='coerce').dt.strftime("%Y/%m/%d")

### 1.4.　売上予測を円に変換

In [30]:
# カンマやGを除去して、"億"/"万"単位を円に換算
def parse_sales(s):
    if pd.isna(s):
        return None
    s = str(s).replace("G", "").replace(",", "")
    if "億" in s:
        return float(Decimal(s.replace("億", "")) * Decimal("1e8"))
    elif "万" in s:
        return float(Decimal(s.replace("万", "")) * Decimal("1e4"))
    else:
        return None

### 1.5. 売上列の整理

In [31]:
# 売上予測（円）を作成して四捨五入
df["売上予測（円）"] = df["売上予測"].apply(parse_sales).round().astype("Int64")

# 元の「売上予測」列を削除
df = df.drop(columns=["売上予測"])

# 確認
df.head()

,タイトル,年月,サービス終了日,売上予測（円）
0,スーパーガンダムロワイヤル,2018/12,2021/07/29,248000000
1,スーパーガンダムロワイヤル,2018/11,2021/07/29,251000000
2,スーパーガンダムロワイヤル,2018/10,2021/07/29,263000000
3,スーパーガンダムロワイヤル,2018/09,2021/07/29,190000000
4,スーパーガンダムロワイヤル,2018/08,2021/07/29,198000000


### 1.6. 整形後データの保存

In [32]:
# 整形後データの保存
df.to_csv("cleaned_output.csv", index=False, encoding="utf-8-sig")

### 2.1. 発売日データの読み込み

In [33]:
# ファイルの読み込み
df_cleaned = pd.read_csv("cleaned_output.csv")  # 売上・年月・終了日など
df_4gamer = pd.read_csv("4gamer_output_corrected.csv")  # 発売日・備考などの情報

### 2.2. 2つのファイルを結合

In [34]:
# タイトルをキーにして内部結合
merged_df = pd.merge(df_cleaned, df_4gamer, on="タイトル", how="inner")

### 2.3. カラムの並べ替え

In [35]:
# 希望のカラム順に並べ替え
columns_order = ["タイトル", "年月", "発売日", "サービス終了日", "売上予測（円）", "備考"]
reordered_df = merged_df[columns_order]

### 2.4. 結合データの保存

In [36]:
# 並べ替えたデータを保存（必要であれば）
reordered_df.to_csv("merged_and_ordered.csv", index=False, encoding="utf-8-sig")